https://huggingface.co/blog/fine-tune-whisper

In [1]:
whisper_model = "openai/whisper-small"#"openai/whisper-medium"#"openai/whisper-large-v3"
lang_id = "af"
model_dir="./afr_south_hollandic_model"
output_dir="./afr_southhollandic_model_output"

In [49]:
from datasets import load_from_disk, Audio, load_dataset, concatenate_datasets, Dataset
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback

In [105]:
dataset_dict = load_dataset("google/fleurs", "af_za")
print(dataset_dict)

C:\Users\Augustijn\anaconda3\lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 1032
    })
    validation: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 198
    })
    test: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 264
    })
})


In [106]:
dataset_dict['train'][0]

{'id': 766,
 'num_samples': 72960,
 'path': 'C:\\Users\\Augustijn\\.cache\\huggingface\\datasets\\downloads\\extracted\\d23995f4a224aa5de9a8afb10ca2a4f6cf9c3b82b4a0de28edbdb962f933af80\\10010386886416577453.wav',
 'audio': {'path': 'train/10010386886416577453.wav',
  'array': array([0.        , 0.        , 0.        , ..., 0.00410163, 0.0040049 ,
         0.00438273]),
  'sampling_rate': 16000},
 'transcription': "families met kinders is nie 'n algemene verskynsel nie maar party koshuise laat hulle in privaat kamers toe",
 'raw_transcription': 'Families met kinders is nie ‘n algemene verskynsel nie, maar party koshuise laat hulle in privaat kamers toe.',
 'gender': 1,
 'lang_id': 0,
 'language': 'Afrikaans',
 'lang_group_id': 3}

In [107]:
def get_total_seconds(samples):
    samples = sum(samples)
    seconds = samples/16000
    return seconds

In [108]:
seconds = get_total_seconds(dataset_dict["train"]["num_samples"])
minutes = seconds/60
hours = seconds/3600

print("hours: ", hours)
print("minutes: ", minutes)

#How much do we need to get to 10 hours of data?
extra_needed = 36000-seconds

hours:  2.8910500000000003
minutes:  173.46300000000002


In [109]:
dataset_dict['train'] = dataset_dict['train'].select_columns(['audio', 'transcription'])
dataset_dict['validation'] = dataset_dict['validation'].select_columns(['audio', 'transcription'])
dataset_dict['test'] = dataset_dict['test'].select_columns(['audio', 'transcription'])
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 1032
    })
    validation: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 198
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 264
    })
})

In [110]:
dataset2 = load_from_disk('dataset_south_hollandic')

In [111]:
dataset2 = dataset2.rename_column('text', 'transcription')
dataset2 = dataset2.rename_column('file', 'audio')

In [112]:
print(dataset2)

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 182
})


In [113]:
dataset2 = dataset2.shuffle(seed=42)

In [114]:
import librosa  

durations = []
for file in dataset2['audio']:
    durations.append(librosa.get_duration(filename=file))

C:\Users\Augustijn\AppData\Local\Temp\ipykernel_5620\4269720119.py:5: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  durations.append(librosa.get_duration(filename=file))


In [115]:
dataset_short = dataset2.filter(lambda example, idx: durations[idx] < 480, with_indices=True)

In [116]:
dataset_short

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 109
})

In [117]:
durations_short = [x for x in durations if x < 480]
sum(durations_short)/3600

8.768290434027781

In [118]:
durations

[484.3278125,
 481.8549375,
 689.9859375,
 482.0168125,
 477.454125,
 241.0986875,
 484.6320625,
 483.4800625,
 484.38875,
 725.8129375,
 244.3119375,
 236.9494375,
 480.0913125,
 482.44875,
 234.899625,
 479.0395625,
 461.0550625,
 480.170625,
 702.57975,
 483.3481875,
 245.5410625,
 239.5645,
 480.4661875,
 474.178,
 482.7745625,
 488.823875,
 241.189875,
 491.6141875,
 236.4823125,
 666.958875,
 243.157375,
 244.919875,
 479.834875,
 479.4474375,
 225.999875,
 476.37075,
 485.6636875,
 482.9725625,
 240.4391875,
 243.611,
 247.6305,
 246.4813125,
 243.0438125,
 246.8218125,
 472.637,
 242.818625,
 236.2258125,
 239.8729375,
 233.872375,
 489.3745,
 480.7928125,
 714.465625,
 239.55775,
 240.10425,
 247.3198125,
 229.461,
 242.6239375,
 244.703,
 484.2498125,
 234.9278125,
 483.6343125,
 482.515625,
 753.638625,
 486.01675,
 482.4858125,
 481.0005625,
 239.8675,
 235.912875,
 476.3946875,
 242.7275625,
 239.059125,
 464.6466875,
 240.8218125,
 482.8186875,
 482.3775625,
 486.9745,
 4

In [119]:
seconds = sum(durations)
minutes = seconds/60
hours = seconds/3600

print("hours: ", hours)
print("minutes: ", minutes)

hours:  19.431233958333326
minutes:  1165.8740374999995


In [120]:
#Take a subset that is about the same length as what we need to get to our target length of 10 hours.
#dataset_sel = dataset2.select(range(2358))



print(extra_needed - sum(durations_short[0:89]))

-36.50643750000745


In [121]:
dataset_sel = dataset_short.select(range(89))

In [122]:
dataset_sel= dataset_sel.cast_column("audio", Audio(sampling_rate=16000))

In [123]:
dataset_dict["train"] = concatenate_datasets([dataset_dict["train"], dataset_sel])

In [124]:
feature_extractor = WhisperFeatureExtractor.from_pretrained(whisper_model)

tokenizer = WhisperTokenizer.from_pretrained(whisper_model, language=lang_id, task="transcribe")

processor = WhisperProcessor.from_pretrained(whisper_model, language=lang_id, task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [125]:
#Skip this step as sampling rate is already correct for FLEURS
#dataset_dict = dataset_dict.cast_column("file", Audio(sampling_rate=16000))

In [126]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

dataset_dict = dataset_dict.map(prepare_dataset, remove_columns=dataset_dict.column_names["train"])

Map:   0%|          | 0/1121 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2195 > 1024). Running this sequence through the model will result in indexing errors


In [127]:
model = WhisperForConditionalGeneration.from_pretrained(whisper_model)
'''
#Freeze the first 16 layers of the model to reduce computational load.
for param in model.model.encoder.layers[:16].parameters():
    param.requires_grad = False
''' 
model.generation_config.language = lang_id
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None    

In [128]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [129]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [130]:
metric = evaluate.load("wer")


In [131]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [132]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,  
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False  # Set this according to your evaluation metric
)

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]
)

In [133]:
trainer.train()

C:\Users\Augustijn\AppData\Roaming\Python\Python310\site-packages\transformers\models\whisper\modeling_whisper.py:697: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


RuntimeError: The size of tensor a (2197) must match the size of tensor b (448) at non-singleton dimension 1

In [50]:
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


('./afr_de_FLEURS_model\\tokenizer_config.json',
 './afr_de_FLEURS_model\\special_tokens_map.json',
 './afr_de_FLEURS_model\\vocab.json',
 './afr_de_FLEURS_model\\merges.txt',
 './afr_de_FLEURS_model\\normalizer.json',
 './afr_de_FLEURS_model\\added_tokens.json')

In [51]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.8152217864990234, 'eval_wer': 29.766444232602478, 'eval_runtime': 62.6941, 'eval_samples_per_second': 3.158, 'eval_steps_per_second': 0.207, 'epoch': 8.0}


In [52]:
test_results = trainer.evaluate(eval_dataset=dataset_dict["test"])
print(test_results)

{'eval_loss': 0.795742392539978, 'eval_wer': 28.05194805194805, 'eval_runtime': 85.4006, 'eval_samples_per_second': 3.091, 'eval_steps_per_second': 0.199, 'epoch': 8.0}
